Setup

In [499]:
# from src.utils import load_and_filter_csv
from utils import load_and_filter_csv
import pandas as pd



datadir = '../data/'
sdis91_filepath = datadir + 'interventions-hebdo-2010-2017.csv'
grippe_filepath = datadir + 'incidence-RDD-3-syndromes-grippaux.csv'
diarrhee_filepath = datadir + 'incidence-RDD-6-syndrome-diarrhee-aigue.csv'
varicelle_filepath = datadir + 'incidence-RDD-7-varicelle.csv'
days_off_filepath = datadir + 'hollidays_and_days_off.csv'
wwo_filepath = datadir + 'wwo_hist.csv'

filter_rdd_files = lambda row: row[1]['geo_name'] == 'ILE-DE-FRANCE' and\
                               201001 <= row[1]['week'] <= 201752

print('Import SDIS data')
sdis91_data = load_and_filter_csv(sdis91_filepath, None, separator=';')

print('Import flu data')
grippe_data = load_and_filter_csv(grippe_filepath, filter_rdd_files)

print('Import diarrhoea data')
diarrhee_data = load_and_filter_csv(diarrhee_filepath, filter_rdd_files)

print('Import chickenpox data')
varicelle_data = load_and_filter_csv(varicelle_filepath, filter_rdd_files)

rdd_datasets = {
    "grippe": grippe_data,
    "diarrhee": diarrhee_data,
    "varicelle": varicelle_data
}

print('Import days_off data')
days_off_data = load_and_filter_csv(days_off_filepath,
                                    lambda row:'2010-01-01' <= row[1]['date'] <= '2017-12-12')

print('Import weather data')
wwo_data = load_and_filter_csv(wwo_filepath, None)

print('Setup complete')

Import SDIS data
Import flu data
Import diarrhoea data
Import chickenpox data
Import days_off data
Import weather data
Setup complete


In [500]:
display(
    sdis91_data.head(),
    sdis91_data.tail(),
)

,ope_code_insee,nb_ope,ope_annee,ope_semaine,ope_categorie,ope_code_postal,ope_nom_commune
0,91001.0,0.0,2010.0,1.0,INCN,91150.0,ABBEVILLE LA RIVIERE
1,91016.0,0.0,2010.0,1.0,INCN,91670.0,ANGERVILLE
2,91017.0,0.0,2010.0,1.0,INCN,91470.0,ANGERVILLIERS
3,91021.0,0.0,2010.0,1.0,INCN,91290.0,ARPAJON
4,91022.0,0.0,2010.0,1.0,INCN,91690.0,ARRANCOURT


,ope_code_insee,nb_ope,ope_annee,ope_semaine,ope_categorie,ope_code_postal,ope_nom_commune
405036,91286.0,59.0,2017.0,52.0,SUAP,91350.0,GRIGNY
405037,91549.0,64.0,2017.0,52.0,SUAP,91700.0,STE GENEVIEVE DES BOIS
405038,91174.0,77.0,2017.0,52.0,SUAP,91100.0,CORBEIL ESSONNES
405039,91377.0,84.0,2017.0,52.0,SUAP,91300.0,MASSY
405040,91228.0,103.0,2017.0,52.0,SUAP,91000.0,EVRY


In [501]:
# sdis91_data cleanup

sdis91_data['yearweek'] = sdis91_data.ope_annee.astype(int).astype(str) + sdis91_data.ope_semaine.apply('{:0>2}'.format)

sdis91_data['yearweek'] = sdis91_data.ope_semaine.apply(
    lambda x: "{:02d}".format(x['ope_semaine']),
    axis=1,
)

# explanation :
# ope_annee.astype(int)
# we get rid of the .0 after the year then convert to string

# ope_semaine.apply('{:0>2}'.format)
# '1' -> '01' (explicitly converting to string)

# we convert both left and right operands to strings or
# else they are summed instead of concatenated

sdis91_data['yearweek'] = sdis91_data['yearweek'].astype(float).astype(int)
# when astype(int) is used directly, it causes the following error:
# invalid literal for int() with base 10 (...)

# drop ope_nom_commune & ope_code_postal, we already have the commune's INSEE code
sdis91_data.drop('ope_nom_commune', axis=1, inplace=True)
sdis91_data.drop('ope_code_postal', axis=1, inplace=True)

# drop annee and semaine because we now have yearweek (ISO8601 Yearweek)
sdis91_data.drop('ope_annee', axis=1, inplace=True)
sdis91_data.drop('ope_semaine', axis=1, inplace=True)

sdis91_data.rename(
    columns={'ope_code_insee': 'insee_commune'},
    inplace=True,
)

sdis91_data['insee_commune'] = sdis91_data['insee_commune'].astype(int)
sdis91_data['nb_ope'] = sdis91_data['nb_ope'].astype(int)

display(
    sdis91_data.head(),
    sdis91_data.tail(),
)

TypeError: <lambda>() got an unexpected keyword argument 'axis'

In [482]:
for dataset_name, dataset in rdd_datasets.items():
    display(
        dataset_name,
        dataset.head(3),
        dataset.tail(3),
    )

'grippe'

,week,indicator,inc,inc_low,inc_up,inc100,inc100_low,inc100_up,geo_insee,geo_name
0,201752,3,48723,38697,58749,402,319,485,11,ILE-DE-FRANCE
1,201751,3,49403,40654,58152,408,336,480,11,ILE-DE-FRANCE
2,201750,3,39953,32596,47310,330,269,391,11,ILE-DE-FRANCE


,week,indicator,inc,inc_low,inc_up,inc100,inc100_low,inc100_up,geo_insee,geo_name
414,201003,3,4816,1817,7815,41,15,67,11,ILE-DE-FRANCE
415,201002,3,9134,4877,13391,78,42,114,11,ILE-DE-FRANCE
416,201001,3,10207,5847,14567,87,50,124,11,ILE-DE-FRANCE


'diarrhee'

,week,indicator,inc,inc_low,inc_up,inc100,inc100_low,inc100_up,geo_insee,geo_name
0,201752,6,14938,9638,20238,123,79,167,11,ILE-DE-FRANCE
1,201751,6,16218,11040,21396,134,91,177,11,ILE-DE-FRANCE
2,201750,6,20995,15775,26215,173,130,216,11,ILE-DE-FRANCE


,week,indicator,inc,inc_low,inc_up,inc100,inc100_low,inc100_up,geo_insee,geo_name
414,201003,6,60448,49467,71429,518,424,612,11,ILE-DE-FRANCE
415,201002,6,61576,48316,74836,527,413,641,11,ILE-DE-FRANCE
416,201001,6,42889,33449,52329,367,286,448,11,ILE-DE-FRANCE


'varicelle'

,week,indicator,inc,inc_low,inc_up,inc100,inc100_low,inc100_up,geo_insee,geo_name
0,201752,7,368,0,920,3,0,8,11,ILE-DE-FRANCE
1,201751,7,799,0,1962,7,0,17,11,ILE-DE-FRANCE
2,201750,7,2228,476,3980,18,4,32,11,ILE-DE-FRANCE


,week,indicator,inc,inc_low,inc_up,inc100,inc100_low,inc100_up,geo_insee,geo_name
414,201003,7,1420,0,3082,12,0,26,11,ILE-DE-FRANCE
415,201002,7,3120,267,5973,27,3,51,11,ILE-DE-FRANCE
416,201001,7,3735,242,7228,32,2,62,11,ILE-DE-FRANCE


In [483]:
# rdd_datasets cleanup

for _, rdd_dataset in rdd_datasets.items():

    # always 11 and "ILE-DE-FRANCE"
    rdd_dataset.drop('geo_name', axis=1, inplace=True)
    rdd_dataset.drop('geo_insee', axis=1, inplace=True)

    # always 3, representing grippe, "Influenza-like illness"
    rdd_dataset.drop('indicator', axis=1, inplace=True)


    # drop these values for now
    rdd_dataset.drop('inc_low', axis=1, inplace=True)
    rdd_dataset.drop('inc_up', axis=1, inplace=True)
    rdd_dataset.drop('inc100', axis=1, inplace=True)
    rdd_dataset.drop('inc100_low', axis=1, inplace=True)
    rdd_dataset.drop('inc100_up', axis=1, inplace=True)

    rdd_dataset.rename(
        columns={'week': 'yearweek'},
        inplace=True,
    )

In [484]:
for dataset_name, dataset in rdd_datasets.items():
    display(
        dataset_name,
        dataset.head(3),
        dataset.tail(3),
    )

'grippe'

,yearweek,inc
0,201752,48723
1,201751,49403
2,201750,39953


,yearweek,inc
414,201003,4816
415,201002,9134
416,201001,10207


'diarrhee'

,yearweek,inc
0,201752,14938
1,201751,16218
2,201750,20995


,yearweek,inc
414,201003,60448
415,201002,61576
416,201001,42889


'varicelle'

,yearweek,inc
0,201752,368
1,201751,799
2,201750,2228


,yearweek,inc
414,201003,1420
415,201002,3120
416,201001,3735


In [485]:
display(
    days_off_data.head(),
    days_off_data.tail(),
)

,date,vacances_zone_a,vacances_zone_b,vacances_zone_c,nom_vacances
0,2010-01-01,True,True,True,Vacances de Noël
1,2010-01-02,True,True,True,Vacances de Noël
2,2010-01-03,True,True,True,Vacances de Noël
3,2010-02-06,False,True,False,Vacances d'hiver
4,2010-02-07,False,True,False,Vacances d'hiver


,date,vacances_zone_a,vacances_zone_b,vacances_zone_c,nom_vacances
1193,2017-11-01,True,True,True,Vacances de la Toussaint
1194,2017-11-02,True,True,True,Vacances de la Toussaint
1195,2017-11-03,True,True,True,Vacances de la Toussaint
1196,2017-11-04,True,True,True,Vacances de la Toussaint
1197,2017-11-05,True,True,True,Vacances de la Toussaint


In [486]:
# days_off_data cleanup

from datetime import datetime

# vacation zone for Ile-De-France : Zone C
days_off_data.drop('vacances_zone_a', axis=1, inplace=True)
days_off_data.drop('vacances_zone_b', axis=1, inplace=True)

# don't need the name
days_off_data.drop('nom_vacances', axis=1, inplace=True)

days_off_data['yearweek'] = days_off_data.apply(
    lambda x:
        # year
        str(datetime.strptime(x['date'], '%Y-%m-%d').year) +

        # plus week number (while adding leading 0 if needed)
        "{:02d}".format(datetime.strptime(x['date'], '%Y-%m-%d').isocalendar()[1]),
        axis=1
)

# date isn't needed anymore, as we now have yearweek
days_off_data.drop('date', axis=1, inplace=True)

days_off_data.rename(
    columns={'vacances_zone_c': 'has_vacation'},
    inplace=True,
)

# days_off_data does not contain a line per week, only a line where it is holiday in at
# least one of the three zones, so now we have some lines where the "has_vacation"
# column is false. It corresponds to when some other zone is in vacation, but not the zone C, we don't need it.
days_off_data.drop(days_off_data[days_off_data.has_vacation == False].index, inplace=True)

# 'not days_off_data.has_vacation' doesn't work (it is ambiguous), thus the use
# of == False

display(
    days_off_data.head(),
    days_off_data.tail(),
)

,has_vacation,yearweek
0,True,201053
1,True,201053
2,True,201053
17,True,201007
18,True,201007


,has_vacation,yearweek
1193,True,201744
1194,True,201744
1195,True,201744
1196,True,201744
1197,True,201744


In [487]:
display(
    wwo_data.head(),
    wwo_data.tail(),
)

,week,temperature_max,temperature_min,temperature_average,total_snow_cm,precipitation_mm,city
0,201752,13.0,0.0,6.71,0.0,7.1,Ondreville-Sur-Essonne
1,201751,9.0,2.0,6.57,0.0,3.6,Ondreville-Sur-Essonne
2,201750,9.0,1.0,4.86,0.0,6.9,Ondreville-Sur-Essonne
3,201749,10.0,0.0,4.57,0.1,1.9,Ondreville-Sur-Essonne
4,201748,9.0,-1.0,3.00,0.0,8.5,Ondreville-Sur-Essonne


,week,temperature_max,temperature_min,temperature_average,total_snow_cm,precipitation_mm,city
412,201005,9.0,-2.0,4.29,0.2,10.5,Ondreville-Sur-Essonne
413,201004,5.0,-5.0,0.43,0.2,0.7,Ondreville-Sur-Essonne
414,201003,7.0,-1.0,2.29,0.0,1.9,Ondreville-Sur-Essonne
415,201002,8.0,-5.0,0.71,1.5,2.5,Ondreville-Sur-Essonne
416,201001,-0.0,-10.0,-4.43,2.6,3.2,Ondreville-Sur-Essonne


In [488]:
# cleanup wwo_data

# city is always "Ondreville-Sur-Essonne"
wwo_data.drop('city', axis=1, inplace=True)

wwo_data.rename(
    columns={'week': 'yearweek'},
    inplace=True,
)

display(
    wwo_data.head(),
    wwo_data.tail(),
)

,yearweek,temperature_max,temperature_min,temperature_average,total_snow_cm,precipitation_mm
0,201752,13.0,0.0,6.71,0.0,7.1
1,201751,9.0,2.0,6.57,0.0,3.6
2,201750,9.0,1.0,4.86,0.0,6.9
3,201749,10.0,0.0,4.57,0.1,1.9
4,201748,9.0,-1.0,3.00,0.0,8.5


,yearweek,temperature_max,temperature_min,temperature_average,total_snow_cm,precipitation_mm
412,201005,9.0,-2.0,4.29,0.2,10.5
413,201004,5.0,-5.0,0.43,0.2,0.7
414,201003,7.0,-1.0,2.29,0.0,1.9
415,201002,8.0,-5.0,0.71,1.5,2.5
416,201001,-0.0,-10.0,-4.43,2.6,3.2


In [489]:
# aggregate everything

# sdis91_data

# grippe_data
# diarrhee_data,
# varicelle_data

# days_off_data
# wwo_data


# grippe_data.join(
#     diarrhee_data,
#     lsuffix='_grippe', rsuffix='_diarrhee',
#     left
# )

diseases_data = grippe_data.merge(
    diarrhee_data, on='yearweek', how='outer'
).merge(
    varicelle_data, on='yearweek', how='outer'
)

diseases_data.rename(
    columns={
        'inc_x': 'inc_grippe',
        'inc_y': 'inc_diarrhee',
        'inc': 'inc_varicelle',
    },
    inplace=True,
)

# merged_data = sdis91_data.merge(
#     diseases_data, on='yearweek', how='outer'
# ).merge(
#     days_off_data, on='yearweek', how='outer'
# ).merge(
#     wwo_data, on='yearweek', how='outer'
# )

sdis91_data

,insee_commune,nb_ope,ope_categorie,yearweek
0,91001,0,INCN,1
1,91016,0,INCN,1
2,91017,0,INCN,1
3,91021,0,INCN,1
4,91022,0,INCN,1
...,...,...,...,...
405036,91286,59,SUAP,52
405037,91549,64,SUAP,52
405038,91174,77,SUAP,52
405039,91377,84,SUAP,52
